### Evaluation utilities

Anything I might possibly reuse in evaluating models that is easily separated will be put here

In [ ]:
import matplotlib
matplotlib.use('svg')  # vector outputs
import matplotlib.pyplot as plt
import os

In [ ]:
def generic_outputs_structure(basepath, model_name, date_str, time_str, **kwargs):
    """
    Makes a default file structure for the outputs of the training. No extensions are returned.
    
    Parameters
    ----------
    basepath : string 
        the full base path to use
    model_name : string
        the name of the model
    date_str : string
        date the script was executed, for subfolder
    time_str : string
        time the script was executed
        
    Outputs
    -------
    log_file : string
        path to log file
    cf_file : string
        path to confusion matrices file
    samplefile_start : string
        the beginning part of each sample file (needs info for each file for time index etc, so this isn't the full eventual filename)
    
    """
    dir_general = basepath+'/'+ date_str
    dir_samples = dir_general + '/' + "samples"
    os.makedirs(dir_general, exist_ok=True)
    os.makedirs(dir_samples, exist_ok=True)
    
    log_file = dir_general + '/' + model_name + 'log' + time_str
    cf_file = dir_general + '/' + model_name + 'cfmatrix' + time_str
    samplefile_start = dir_samples + '/' + model_name + time_str
    
    return log_file, cf_file, samplefile_start
    
    

In [ ]:
def plt_traintest_cf_matrices(train_true, train_pred, test_true, test_pred, filename):
    """
    Plots confusion matrices normalized both ways given true and predicted data for 
    training and test data.
    
    Parameters
    ----------
    train_true : 1d array, length train_batch_size
        true labels for the training batch
    train_pred : 1d array, length train_batch_size
        pred labels for the training batch
    test_true : 1d array, length test_batch_size
        true labels for the testing batch
    test_pred : 1d array, length test_batch_size
        pred labels for the testing batch
    filename : string
        The full file name WITHOUT extension. 
    
    Returns
    -------
    None
    
    """
    fig, ax = plt.subplots(2,2)
    ax[0,0].set(title="Training Confusion, normalized true")
    ax[0,1].set(title="Testing Confusion, normalized true")
    ax[1,0].set(title="Training Confusion, normalized pred")
    ax[1,1].set(title="Testing Confusion, normalized pred")
    cf_train_t = ConfusionMatrixDisplay(confusion_matrix(train_true, train_pred, normalize='true'))
    cf_test_t = ConfusionMatrixDisplay(confusion_matrix(test_true, test_pred, normalize='true'))
    cf_train_p = ConfusionMatrixDisplay(confusion_matrix(train_true, train_pred, normalize='pred'))
    cf_test_p = ConfusionMatrixDisplay(confusion_matrix(test_true, test_pred, normalize='pred'))
    cf_train_t.plot(ax=ax[0,0])
    cf_test_t.plot(ax=ax[0,1])
    cf_train_p.plot(ax=ax[1,0])
    cf_test_p.plot(ax=ax[1,1])

    fig.tight_layout()
    fig.savefig(filename+".svg")
    plt.close(fig='all')
    

In [ ]:
def show_2d_success(predictions, xvals, zvals, flux_fn_mesh, flux_fn, filename):
    """
    Plot predictions onto the 2d flux function contour
    
    Parameters
    ----------
    predictions : array-like, 1d
        predicted categories at a list of points
    xvals : array-like, 1d
        x locations of each point
    zvals: : array-like, 1d
        z locations of each point
    flux_fn_mesh : list of 1d array-like
        the mesh for the flux function in Statmeshvar style format
    flux_fn : array-like, 2d
        array holding the flux function
    filename : string
        The full file name WITHOUT extension. 
        
    Returns
    -------
    None
    
    """
    cmap = ['r','b','g','y']
    colors = [cmap[i] for i in predictions]
    
    X,Y = np.meshgrid(*flux_fn_mesh, indexing='ij')
    fig, ax = plt.subplots(figsize=(15,10))
    ctr =ax.contour(X,Y,flux_fn, levels=100, zorder=1)
    ax.set(title="Predicted structures over flux contours")
    plt.colorbar(mappable=ctr)
    ax.scatter(xvals, zvals, color=colors, zorder=2, s=.01, marker=".")
    
    fig.tight_layout()
    fig.savefig(filename+".svg")
    plt.close(fig='all')
    

In [ ]:
def plot_sample(inputs, input_names, mesh, true, pred, filename, inputs_padding=0, true_coords=None):
    """
    Plots a representation of a given sample.
    
    Parameters
    ----------
    inputs : list of array_likes
        list of the inputs to the model for the sample. Each element of the interior list must have first dimension length (padded_segment_length)
    input_names : list of strings
        labels for each of the inputs to display on the summary plot
    mesh : 1d array-like
        1d mesh for the padded segments of the sample
    true : array-like
        array of the true value arrays for the sample, ONE-HOT
    pred : array-like
       array of the predicted value arrays for the sample, PROBABILITY DIST
    filename : string
        desired file to vomit the plots into (sans extension). Absolute or relative path.
    inputs_padding : integer, default 0
        the number of points added to each sample for the inputs (when compared to the prediction region)
    true_coords : array-like of shape (segment_length, ndim), default None
        the coordinates in n-dimensional "real space" of each point
    
    Returns
    -------
    None
    """
    num_inputs = len(inputs)
    true_num = np.argmax(true, axis=1) # to integer categories
    pred_num = np.argmax(pred, axis=1)
    pred_probability = np.amax(pred, axis=1)
    # setting up the figure
    num_plots = num_inputs + 2
    if true_coords is not None:
        num_plots += 2
    fig, ax = plt.subplots(num_plots, figsize=(10,15)) 
    for axis in ax:
        axis.set_xlim(mesh[0], mesh[-1])
    # plot the inputs    
    for i in range(num_inputs): 
        ax[i].plot(mesh,inputs[i])
        ax[i].set(ylabel=input_names[i])
    # plot the results    
    ax[num_inputs].set(ylabel="Category number")
    ax[num_inputs].scatter(mesh[inputs_padding:-inputs_padding], true_num, s=18, marker='o', c='b', label='true')
    ax[num_inputs].scatter(mesh[inputs_padding:-inputs_padding], pred_num, s=6, marker='o', c='r', label='pred')  
    ax[num_inputs].legend()
    # plot confidences
    ax[num_inputs+1].set(ylabel="Model-determined\npercent chance correct")
    ax[num_inputs+1].plot(mesh[inputs_padding:-inputs_padding], pred_probability)  
    # if applicable plot "real" locations
    if true_coords is not None:
        ax[-2].plot(mesh[inputs_padding:-inputs_padding], true_coords[:,0])
        ax[-2].set(ylabel="True x coordinate")
        ax[-1].plot(mesh[inputs_padding:-inputs_padding], true_coords[:,1])
        ax[-1].set(ylabel="True z coordinate")
        
    fig.tight_layout()
    fig.savefig(filename+".svg")
    plt.close(fig='all')


In [ ]:
def plot_reps(inputs, input_names, meshes, trues, preds, basepath, inputs_padding=0, true_coords=None, exs_per_cat=5 ):
    """
    Take a representative sample of the samples in a batch, and plot them.
    Representative sample needs successful and failed samples of each category
    
    Parameters
    ----------
    inputs : list of array-likes
        list of the inputs to the model for each sample. Each element of the interior list must have first dimension length (batch_size)
    input_names : list of strings
        labels for each of the inputs to display on the summary plot
    meshes : list of array-likes
        1d mesh for the padded segments of each sample
    trues : array-like
        array of the true value arrays for each sample ONE-HOT (batch, segment_length, num_cats)
    preds : array-like
       array of the predicted value arrays for each sample PROBABILITY DIST
    basepath : 
        desired file folder to vomit the plots into, plus any beginning you want each file to have
        e.g. "/home/run_31_"
    inputs_padding : integer, default 0
        the number of points added to each sample for the inputs (when compared to the prediction region)
    true_coords : array-like of shape (batch_size, segment_length, ndim)
        the coordinates in n-dimensional "real space" of each point
    exs_per_cat : integer, default 5
        the number of samples to use that contain each categorization possiblity (predicted category a, true category b)
    
    Returns
    -------
    None
    """
    rng = np.random.default_rng(12)
    # Populations to sample: 
    #     every combo of true/pred categorization.
    preds_oh = np.apply_along_axis(lambda x: np.array(x == max(x), dtype=int), -1, preds)
    # get all combinations of (cat1, cat2) 
    cats = [i for i in range(preds.shape[-1])]
    cat_pairs = np.stack(np.meshgrid(cats,cats), axis=-1).reshape(-1,2)
    # for each combo, find the samples in the batch for which there exists an element with true=cat1 AND pred=cat2
    #    np.any(np.logical_and(true == cat1, pred == cat2), axis=1)
    for i in range(cat_pairs.shape[0]):
        valid_population_idxs = np.nonzero(np.any(np.logical_and(trues[...,cat_pairs[i,0]], preds[...,cat_pairs[i,1]]), axis=-1))[0]
        repr_sample_idxs = rng.choice(valid_population_idxs, min(exs_per_cat, len(valid_population_idxs)), replace=False)
        for j in repr_sample_idxs:
            filename=basepath+f"sample_{j}"
            coords = true_coords[j] if true_coords is not None else None  # don't try to index None, python doesn't like that
            plot_sample([input[j] for input in inputs], input_names, meshes[j], trues[j], preds[j], filename,
                        inputs_padding=inputs_padding, true_coords = coords)
    
    
    